In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
from knapsack_problem.knapsack import best_full_teams
from knapsack_problem.player import Player
import pandas as pd
import math

In [3]:
DATA_PATH = "predictions/preds_next_season.csv"

In [4]:
df = pd.read_csv(DATA_PATH).rename(columns={"Pred": "Prediction"})

In [5]:
df.columns

Index(['Name_original', 'GW', 'Season', 'Team', 'Opponent', 'Was_home',
       'Team_rating', 'Opp_rating', 'FPL_pos', 'Price', 'Prediction'],
      dtype='object')

In [6]:
df = df[df["GW"] == 3]

In [7]:
df.head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Prediction
50,Erling Haaland,3.0,2023-24,Manchester City,Sheffield United,0.0,2080.850586,1637.473755,FWD,140.0,5.586223
64,Mohamed Salah,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,MID,125.0,5.561158
88,Bukayo Saka,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,86.0,5.444934
97,Gabriel Martinelli Silva,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,80.0,5.306480
224,Bruno Borges Fernandes,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,MID,85.0,4.645544
273,Callum Wilson,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1887.064819,1943.976074,FWD,80.0,4.627954
317,Andrew Robertson,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,DEF,65.0,4.471224
394,Solly March,3.0,2023-24,Brighton,West Ham,1.0,1832.364258,1777.551270,MID,65.0,4.388664
413,Martin Ødegaard,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,85.0,4.374848
447,Alisson Ramses Becker,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,GK,55.0,4.360530


In [8]:
df = df[(df["Prediction"] > 3.5) | (df["FPL_pos"] == "GK")]

In [9]:
df.shape, df[df["FPL_pos"] == "GK"].shape, df[df["FPL_pos"] == "DEF"].shape, df[df["FPL_pos"] == "MID"].shape, df[df["FPL_pos"] == "FWD"].shape

((82, 11), (40, 11), (13, 11), (22, 11), (7, 11))

In [10]:
df.head()

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Prediction
50,Erling Haaland,3.0,2023-24,Manchester City,Sheffield United,0.0,2080.850586,1637.473755,FWD,140.0,5.586223
64,Mohamed Salah,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,MID,125.0,5.561158
88,Bukayo Saka,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,86.0,5.444934
97,Gabriel Martinelli Silva,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,80.0,5.306480
224,Bruno Borges Fernandes,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,MID,85.0,4.645544


In [11]:
playersDB = []
for i, row in df.iterrows():
    playersDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [12]:

for p in playersDB:
    print(p.__str__())

(Erling Haaland, FWD, 140.0, 5.59, Manchester City) - (fixtures: 1.0)
(Mohamed Salah, MID, 125.0, 5.56, Liverpool) - (fixtures: 1.0)
(Bukayo Saka, MID, 86.0, 5.44, Arsenal) - (fixtures: 1.0)
(Gabriel Martinelli Silva, MID, 80.0, 5.31, Arsenal) - (fixtures: 1.0)
(Bruno Borges Fernandes, MID, 85.0, 4.65, Manchester Utd) - (fixtures: 1.0)
(Callum Wilson, FWD, 80.0, 4.63, Newcastle Utd) - (fixtures: 1.0)
(Andrew Robertson, DEF, 65.0, 4.47, Liverpool) - (fixtures: 1.0)
(Solly March, MID, 65.0, 4.39, Brighton) - (fixtures: 1.0)
(Martin Ødegaard, MID, 85.0, 4.37, Arsenal) - (fixtures: 1.0)
(Alisson Ramses Becker, GK, 55.0, 4.36, Liverpool) - (fixtures: 1.0)
(David Raya Martin, GK, 50.0, 4.35, Arsenal) - (fixtures: 1.0)
(Aaron Ramsdale, GK, 50.0, 4.33, Arsenal) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 80.0, 4.31, Liverpool) - (fixtures: 1.0)
(Harvey Barnes, MID, 65.0, 4.21, Newcastle Utd) - (fixtures: 1.0)
(William Saliba, DEF, 51.0, 4.15, Arsenal) - (fixtures: 1.0)
(Antony Matheus dos 

In [13]:
# 2 example Goalkeepers
# playersDB.append(Player("GK1", "GK", 5, 4, "Manchester Utd"))
# playersDB.append(Player("GK2", "GK", 4, 3, "Nott'ham Forest"))

In [14]:
possible_formations = [
    [3, 4, 3],
    [3, 5, 2],
    [4, 3, 3],
    [4, 4, 2],
    [4, 5, 1],
    [5, 3, 2],
    [5, 4, 1],
]

In [15]:
budget = 1000

In [16]:
%%time
best_team = best_full_teams(playersDB, possible_formations, budget)

With formation [3, 5, 2]: 52.86  | (price = 912.0)
(Alisson Ramses Becker, GK, 55.0, 4.36, Liverpool) - (fixtures: 1.0)
(Andrew Robertson, DEF, 65.0, 4.47, Liverpool) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 80.0, 4.31, Liverpool) - (fixtures: 1.0)
(William Saliba, DEF, 51.0, 4.15, Arsenal) - (fixtures: 1.0)
(Mohamed Salah, MID, 125.0, 5.56, Liverpool) - (fixtures: 1.0)
(Bukayo Saka, MID, 86.0, 5.44, Arsenal) - (fixtures: 1.0)
(Gabriel Martinelli Silva, MID, 80.0, 5.31, Arsenal) - (fixtures: 1.0)
(Bruno Borges Fernandes, MID, 85.0, 4.65, Manchester Utd) - (fixtures: 1.0)
(Solly March, MID, 65.0, 4.39, Brighton) - (fixtures: 1.0)
(Erling Haaland, FWD, 140.0, 5.59, Manchester City) - (fixtures: 1.0)
(Callum Wilson, FWD, 80.0, 4.63, Newcastle Utd) - (fixtures: 1.0)


With formation [4, 4, 2]: 52.61  | (price = 912.0)
(Alisson Ramses Becker, GK, 55.0, 4.36, Liverpool) - (fixtures: 1.0)
(Andrew Robertson, DEF, 65.0, 4.47, Liverpool) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 80.

In [24]:
f"Best team score: {best_team[0][1]}"

'Best team score: 52.86'

In [18]:
def best_team_to_df(best_team_squad, df, custom_fpl_order):
    names = [p.name for p in best_team_squad]
    
    # Convert "FPL_pos" column to categorical with custom order
    df["FPL_pos"] = pd.Categorical(df["FPL_pos"], categories=custom_fpl_order, ordered=True)
    
    # Filter by names and sort by custom ordered "FPL_pos" and "Price"
    return df[df["Name_original"].isin(names)].sort_values(by=["FPL_pos", "Price"], ascending=False)

# Define your custom order for FPL_pos
custom_fpl_order = ["FWD", "MID", "DEF", "GK"]

In [19]:
best_squad = best_team_to_df(best_team[0][2], df, custom_fpl_order)
best_squad

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Prediction
447,Alisson Ramses Becker,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,GK,55.0,4.360530
526,Trent Alexander-Arnold,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,DEF,80.0,4.308818
317,Andrew Robertson,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,DEF,65.0,4.471224
594,William Saliba,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,DEF,51.0,4.148964
64,Mohamed Salah,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,MID,125.0,5.561158
88,Bukayo Saka,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,86.0,5.444934
224,Bruno Borges Fernandes,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,MID,85.0,4.645544
97,Gabriel Martinelli Silva,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,80.0,5.306480
394,Solly March,3.0,2023-24,Brighton,West Ham,1.0,1832.364258,1777.551270,MID,65.0,4.388664
50,Erling Haaland,3.0,2023-24,Manchester City,Sheffield United,0.0,2080.850586,1637.473755,FWD,140.0,5.586223


In [20]:
best_squad.to_csv("predictions/best_squad.csv")